### Plot Velocities and streamlines

In [ ]:
import numpy as np
import pointAdvection
import pointCollection as pc
import matplotlib.pyplot as plt
import matplotlib.colors as colors
#%matplotlib widget
%load_ext autoreload
%autoreload 2

### File locations

In [ ]:
MOA_file = 'moa125_2014_hp1_v01.tif'
# https://nsidc.org/data/NSIDC-0484
vel_file = 'antarctica_ice_velocity_450m_v2.nc'

#### Set bounds for Larsen C

In [ ]:
# grid bounds to read
xlimits = (-2470000, -2050000)
ylimits = (895000, 1325000)

#### Pick a location in the center of the map

In [ ]:
xc, yc = [-2196826.298701299, 1150836.038961039]

#### Read the velocity data

In [ ]:
d2 = pointAdvection.advection().from_nc(vel_file,
    bounds=[xlimits,ylimits], scale=1.0)
# read in the MOA
MOA = pc.grid.data().from_geotif(MOA_file,
    bounds=[xlimits,ylimits])

#### Move the points downstream


In [ ]:
# pick some points around the center point
dx, dy = np.meshgrid([-1.e4, 0, 1.e4], [-1.e4, 0, 1.e4])
# convert to arrays
d2.x = xc + dx.ravel()
d2.y = yc + dy.ravel()
d2.t = (2015.0-2000) + np.zeros_like(dx).ravel()
# translate points
d2.translate_parcel(t0=(2100.0-2000), streak=True, step=1);

#### Plot the results

In [ ]:
plt.figure(); 
MOA.show(clim=[0, 16386], cmap='gray')
cmap = pointAdvection.tools.custom_colormap(180, 'Rignot')
plt.imshow(np.abs(d2.velocity.U+1j*d2.velocity.V),
    extent=d2.velocity.extent, origin='lower',
    cmap=cmap, norm=colors.LogNorm(vmin=1, vmax=3000),
    alpha=0.75
)
plt.colorbar()
plt.plot(d2.x, d2.y, 'w*')
plt.plot(d2.streak['x'], d2.streak['y'], marker='.', ms=1);